In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

In [3]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Saint George,1,US,1627559120,41,37.1041,-113.5841,87.64,1.01
1,Comodoro Rivadavia,0,AR,1627559460,49,-45.8667,-67.5000,48.06,8.05
2,Fortuna,75,US,1627559053,95,40.5982,-124.1573,52.66,0.00
3,Hithadhoo,100,MV,1627559103,72,-0.6000,73.0833,82.22,9.93
4,Talakan,100,RU,1627559506,93,50.3167,130.3667,69.71,3.53
...,...,...,...,...,...,...,...,...,...
1026,La Paz,69,BO,1627560407,72,-16.5000,-68.1500,41.90,1.43
1027,Pemangkat,27,ID,1627560408,82,1.1667,108.9667,81.88,13.02
1028,Otane,4,NZ,1627560409,94,-39.8833,176.6333,41.02,2.04
1029,Amga,0,RU,1627560410,59,60.8953,131.9608,56.32,4.52


In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
max_intensity

100

In [7]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
150,Kostanay,0,KZ,1627559597,31,53.1667,63.5833,73.69,8.95
210,Richards Bay,0,ZA,1627559297,23,-28.7830,32.0377,77.58,6.17
257,Belyy Yar,0,RU,1627559293,43,53.6039,91.3903,73.29,6.71
346,Cayenne,0,GF,1627559758,83,4.9333,-52.3333,78.84,2.30
377,Hermiston,0,US,1627559784,60,45.8404,-119.2895,74.61,3.78
468,Arrondissement de La Rochelle,0,FR,1627559869,49,46.2500,-1.0000,74.61,6.91
624,Kostanay,0,KZ,1627559597,31,53.1667,63.5833,73.69,8.95
684,Richards Bay,0,ZA,1627560161,23,-28.7830,32.0377,77.58,6.17
731,Belyy Yar,0,RU,1627560224,43,53.6039,91.3903,73.29,6.71
820,Cayenne,0,GF,1627559758,83,4.9333,-52.3333,78.84,2.30


In [8]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
150,Kostanay,KZ,53.1667,63.5833,
210,Richards Bay,ZA,-28.7830,32.0377,
257,Belyy Yar,RU,53.6039,91.3903,
346,Cayenne,GF,4.9333,-52.3333,
377,Hermiston,US,45.8404,-119.2895,
468,Arrondissement de La Rochelle,FR,46.2500,-1.0000,
624,Kostanay,KZ,53.1667,63.5833,
684,Richards Bay,ZA,-28.7830,32.0377,
731,Belyy Yar,RU,53.6039,91.3903,
820,Cayenne,GF,4.9333,-52.3333,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 150: Kostanay.
Missing field/result... skipping.
------------
Retrieving Results for Index 210: Richards Bay.
Missing field/result... skipping.
------------
Retrieving Results for Index 257: Belyy Yar.
Missing field/result... skipping.
------------
Retrieving Results for Index 346: Cayenne.
Missing field/result... skipping.
------------
Retrieving Results for Index 377: Hermiston.
Missing field/result... skipping.
------------
Retrieving Results for Index 468: Arrondissement de La Rochelle.
Missing field/result... skipping.
------------
Retrieving Results for Index 624: Kostanay.
Missing field/result... skipping.
------------
Retrieving Results for Index 684: Richards Bay.
Missing field/result... skipping.
------------
Retrieving Results for Index 731: Belyy Yar.
Missing field/result... skipping.
------------
Retrieving Results for Index 820: Cayenne.
Missing field/result... skipping.
------------
Retrieving Results for Index 851: Hermiston.
Missing field/r

In [11]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
150,Kostanay,KZ,53.1667,63.5833,
210,Richards Bay,ZA,-28.7830,32.0377,
257,Belyy Yar,RU,53.6039,91.3903,
346,Cayenne,GF,4.9333,-52.3333,
377,Hermiston,US,45.8404,-119.2895,
468,Arrondissement de La Rochelle,FR,46.2500,-1.0000,
624,Kostanay,KZ,53.1667,63.5833,
684,Richards Bay,ZA,-28.7830,32.0377,
731,Belyy Yar,RU,53.6039,91.3903,
820,Cayenne,GF,4.9333,-52.3333,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))